In [0]:
import requests

In [0]:
def send_message(message):
    TOKEN = dbutils.secrets.get("job_notification", "Telegram_Token")
    CHAT_ID = ["5910263658", "7348273044"]
    MESSAGE = message

    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    for user in CHAT_ID:
        payload = {
            "chat_id": user,
            "text": MESSAGE,
            "parse_mode": "HTML"
        }

        res = requests.post(url, data=payload)

In [0]:
df = spark.table("staging_catalogue.jobs.linkedJobs")
df = df.orderBy(df["score"].desc())

In [0]:
s = ''
count = 1
tc = 1
tl = df.count()
for row in df.collect():
    # print(row)
    s += f"""{tc}. <a href = "{row["url"]}" >{row["title"]}</a>\n<b>{row["company"]}</b>, <i>{row["location"]}, {row["hrs_ago"]}</i> \n\n"""
    if count==20 or tc==tl:
        send_message(s)
        print(s)
        count = 0
        s = ''
    count+=1
    tc+=1

In [0]:
row['location']